In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime, timedelta

pd.set_option('max_columns', None)
pd.set_option('max_rows', 50)

# Weather Data

### Read Data from CSV

In [2]:
wx2016 = pd.read_csv('data/wxkslc.csv', low_memory=False)
wx2016 = wx2016[1:]
wx2017 = pd.read_csv('data/wxkslc2017.csv', low_memory=False)
wx2017 = wx2017[1:]

In [3]:
raw = pd.concat([wx2016,wx2017])
raw.head()

,Station_ID,Date_Time,altimeter_set_1,air_temp_set_1,dew_point_temperature_set_1,relative_humidity_set_1,wind_speed_set_1,wind_direction_set_1,wind_gust_set_1,snow_depth_set_1,sea_level_pressure_set_1,weather_cond_code_set_1,cloud_layer_3_code_set_1,pressure_tendency_set_1,qc_set_1,precip_accum_one_hour_set_1,precip_accum_three_hour_set_1,metar_origin_set_1,cloud_layer_1_code_set_1,cloud_layer_2_code_set_1,precip_accum_six_hour_set_1,precip_accum_24_hour_set_1,visibility_set_1,metar_remark_set_1,metar_set_1,air_temp_high_6_hour_set_1,air_temp_low_6_hour_set_1,peak_wind_speed_set_1,ceiling_set_1,pressure_change_code_set_1,air_temp_high_24_hour_set_1,air_temp_low_24_hour_set_1,peak_wind_direction_set_1,dew_point_temperature_set_1d,wind_chill_set_1d,wind_cardinal_direction_set_1d,pressure_set_1d,sea_level_pressure_set_1d,heat_index_set_1d,weather_condition_set_1d,weather_condition_set_2d,weather_condition_set_3d
1,KSLC,2016-01-01T00:00:00Z,30.45,12.02,6.08,76.51,9.0,360.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.98,2.9,N,883.05,1027.38,NaN,Clear,NaN,NaN
2,KSLC,2016-01-01T00:05:00Z,30.45,12.02,6.08,76.51,7.0,360.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.98,NaN,N,883.05,1027.38,NaN,Clear,NaN,NaN
3,KSLC,2016-01-01T00:10:00Z,30.45,12.02,6.08,76.51,7.0,360.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.98,NaN,N,883.05,1027.38,NaN,Clear,NaN,NaN
4,KSLC,2016-01-01T00:15:00Z,30.45,12.02,5.0,72.82,7.0,360.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.88,NaN,N,883.05,1027.41,NaN,Clear,NaN,NaN
5,KSLC,2016-01-01T00:20:00Z,30.45,10.94,5.0,76.4,7.0,350.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.9,NaN,N,883.05,1028.01,NaN,Clear,NaN,NaN


### Select Relevant Features

In [4]:
selectCol = raw[['Date_Time','air_temp_set_1','dew_point_temperature_set_1d',
                 'wind_speed_set_1','wind_direction_set_1','wind_gust_set_1',
                 'snow_depth_set_1','cloud_layer_1_code_set_1',
                 'cloud_layer_2_code_set_1','cloud_layer_3_code_set_1',
                 'visibility_set_1','ceiling_set_1','weather_condition_set_1d',
                 'weather_condition_set_2d','weather_condition_set_3d']]

In [5]:
selectRow = selectCol.iloc[1:]
selectRow.head()

,Date_Time,air_temp_set_1,dew_point_temperature_set_1d,wind_speed_set_1,wind_direction_set_1,wind_gust_set_1,snow_depth_set_1,cloud_layer_1_code_set_1,cloud_layer_2_code_set_1,cloud_layer_3_code_set_1,visibility_set_1,ceiling_set_1,weather_condition_set_1d,weather_condition_set_2d,weather_condition_set_3d
2,2016-01-01T00:05:00Z,12.02,5.98,7.0,360.0,NaN,NaN,1.0,NaN,NaN,10.0,NaN,Clear,NaN,NaN
3,2016-01-01T00:10:00Z,12.02,5.98,7.0,360.0,NaN,NaN,1.0,NaN,NaN,10.0,NaN,Clear,NaN,NaN
4,2016-01-01T00:15:00Z,12.02,4.88,7.0,360.0,NaN,NaN,1.0,NaN,NaN,10.0,NaN,Clear,NaN,NaN
5,2016-01-01T00:20:00Z,10.94,4.9,7.0,350.0,NaN,NaN,1.0,NaN,NaN,10.0,NaN,Clear,NaN,NaN
6,2016-01-01T00:25:00Z,10.94,4.9,7.0,350.0,NaN,NaN,1.0,NaN,NaN,10.0,NaN,Clear,NaN,NaN


### Dealing with Missing Values

First take a look at the rows with most values missing and the missing value rate for each feature.

In [6]:
#rows with most features missing
mostNaN = selectRow.loc[selectRow.apply(lambda x:x.isnull().sum()>=14, axis=1)]
mostNaN.head()

,Date_Time,air_temp_set_1,dew_point_temperature_set_1d,wind_speed_set_1,wind_direction_set_1,wind_gust_set_1,snow_depth_set_1,cloud_layer_1_code_set_1,cloud_layer_2_code_set_1,cloud_layer_3_code_set_1,visibility_set_1,ceiling_set_1,weather_condition_set_1d,weather_condition_set_2d,weather_condition_set_3d
19864,2016-03-09T16:30:00Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51919,2016-07-29T14:30:00Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52021,2016-07-29T22:20:00Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52031,2016-07-29T23:05:00Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52090,2016-07-30T03:40:00Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#missing value rate
missingRateBefore = selectRow.apply(lambda x: x.isnull().sum()/x.size, axis=0)
missingRateBefore

Date_Time                       0.000000
air_temp_set_1                  0.000721
dew_point_temperature_set_1d    0.000721
wind_speed_set_1                0.005148
wind_direction_set_1            0.014866
wind_gust_set_1                 0.916903
snow_depth_set_1                0.997289
cloud_layer_1_code_set_1        0.004473
cloud_layer_2_code_set_1        0.781371
cloud_layer_3_code_set_1        0.902752
visibility_set_1                0.001493
ceiling_set_1                   0.694139
weather_condition_set_1d        0.004441
weather_condition_set_2d        0.970714
weather_condition_set_3d        0.998870
dtype: float64

For **Date_Time**, there are no missing values.

In [8]:
perfect = selectRow['Date_Time']
perfect.head()

2    2016-01-01T00:05:00Z
3    2016-01-01T00:10:00Z
4    2016-01-01T00:15:00Z
5    2016-01-01T00:20:00Z
6    2016-01-01T00:25:00Z
Name: Date_Time, dtype: object

For features that **have value in most of the rows**, use the strategy that replacing the missing value with the last valid observation in the time series.

In [9]:
common_raw = selectRow[['air_temp_set_1','dew_point_temperature_set_1d','wind_speed_set_1',
                        'wind_direction_set_1','cloud_layer_1_code_set_1','visibility_set_1',
                        'weather_condition_set_1d']]
common = common_raw.fillna(method='ffill')
common.head()

,air_temp_set_1,dew_point_temperature_set_1d,wind_speed_set_1,wind_direction_set_1,cloud_layer_1_code_set_1,visibility_set_1,weather_condition_set_1d
2,12.02,5.98,7.0,360.0,1.0,10.0,Clear
3,12.02,5.98,7.0,360.0,1.0,10.0,Clear
4,12.02,4.88,7.0,360.0,1.0,10.0,Clear
5,10.94,4.9,7.0,350.0,1.0,10.0,Clear
6,10.94,4.9,7.0,350.0,1.0,10.0,Clear


In [10]:
common1 = common.iloc[:,0:-1]
common1.head()

,air_temp_set_1,dew_point_temperature_set_1d,wind_speed_set_1,wind_direction_set_1,cloud_layer_1_code_set_1,visibility_set_1
2,12.02,5.98,7.0,360.0,1.0,10.0
3,12.02,5.98,7.0,360.0,1.0,10.0
4,12.02,4.88,7.0,360.0,1.0,10.0
5,10.94,4.9,7.0,350.0,1.0,10.0
6,10.94,4.9,7.0,350.0,1.0,10.0


In [11]:
common2 = common.iloc[:,-1]
common2.head()

2    Clear
3    Clear
4    Clear
5    Clear
6    Clear
Name: weather_condition_set_1d, dtype: object

For features that **have value missing in most of the rows**, use different strategy respectively.

In [12]:
rare = selectRow[['wind_gust_set_1','snow_depth_set_1','cloud_layer_2_code_set_1',
                  'cloud_layer_3_code_set_1','ceiling_set_1','weather_condition_set_2d',
                  'weather_condition_set_3d']]
rare.head()

,wind_gust_set_1,snow_depth_set_1,cloud_layer_2_code_set_1,cloud_layer_3_code_set_1,ceiling_set_1,weather_condition_set_2d,weather_condition_set_3d
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN


For **Wind Gust** and **Snow Depth**, replace with 0. 

In [13]:
rare1 = rare[['wind_gust_set_1','snow_depth_set_1']].fillna(0)
rare1.head()

,wind_gust_set_1,snow_depth_set_1
2,0,0
3,0,0
4,0,0
5,0,0
6,0,0


For **Cloud**, replace with a value great than the maximum (which means the clouds are too high to affect flight operations).

In [14]:
pd.to_numeric(common['cloud_layer_1_code_set_1']).describe()

count    215049.000000
mean        317.583797
std         475.023856
min           0.000000
25%           1.000000
50%           1.000000
75%         606.000000
max        2506.000000
Name: cloud_layer_1_code_set_1, dtype: float64

In [15]:
pd.to_numeric(rare['cloud_layer_2_code_set_1']).describe()

count    47016.000000
mean       965.676770
std        575.792193
min         62.000000
25%        552.000000
50%        856.000000
75%       1206.000000
max       2806.000000
Name: cloud_layer_2_code_set_1, dtype: float64

In [16]:
pd.to_numeric(rare['cloud_layer_3_code_set_1']).describe()

count    20913.000000
mean      1246.963707
std        681.871106
min        114.000000
25%        704.000000
50%       1103.000000
75%       2002.000000
max       3006.000000
Name: cloud_layer_3_code_set_1, dtype: float64

In [17]:
rare2 = rare[['cloud_layer_2_code_set_1','cloud_layer_3_code_set_1']].fillna(5000)
rare2.head()

,cloud_layer_2_code_set_1,cloud_layer_3_code_set_1
2,5000,5000
3,5000,5000
4,5000,5000
5,5000,5000
6,5000,5000


For **Ceiling**, replace with maximum.

In [18]:
maxVal = pd.to_numeric(rare['ceiling_set_1']).max()

In [19]:
rare3 = rare[['ceiling_set_1']].fillna(maxVal)
rare3.head()

,ceiling_set_1
2,30000
3,30000
4,30000
5,30000
6,30000


For Weather Condition, do nothing.

In [20]:
rare4 = rare[['weather_condition_set_2d','weather_condition_set_3d']]
rare4.head()

,weather_condition_set_2d,weather_condition_set_3d
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
5,NaN,NaN
6,NaN,NaN


### Feature Encoding 

Change date and time features to **DateTime** format.

In [21]:
perfect_dt = perfect.apply(lambda x:datetime.strptime(x,'%Y-%m-%dT%H:%M:%SZ'))
perfect_dt.head()

2   2016-01-01 00:05:00
3   2016-01-01 00:10:00
4   2016-01-01 00:15:00
5   2016-01-01 00:20:00
6   2016-01-01 00:25:00
Name: Date_Time, dtype: datetime64[ns]

Change numeric features to **float** format.

In [22]:
common1_num = common1.apply(pd.to_numeric, errors='raise')
common1_num.head()

,air_temp_set_1,dew_point_temperature_set_1d,wind_speed_set_1,wind_direction_set_1,cloud_layer_1_code_set_1,visibility_set_1
2,12.02,5.98,7.0,360.0,1.0,10.0
3,12.02,5.98,7.0,360.0,1.0,10.0
4,12.02,4.88,7.0,360.0,1.0,10.0
5,10.94,4.90,7.0,350.0,1.0,10.0
6,10.94,4.90,7.0,350.0,1.0,10.0


In [23]:
rare1_num = rare1.apply(pd.to_numeric, errors='raise')
rare1_num.head()

,wind_gust_set_1,snow_depth_set_1
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
5,0.0,0.0
6,0.0,0.0


In [24]:
rare2_num = rare2.apply(pd.to_numeric, errors='raise')
rare2_num.head()

,cloud_layer_2_code_set_1,cloud_layer_3_code_set_1
2,5000.0,5000.0
3,5000.0,5000.0
4,5000.0,5000.0
5,5000.0,5000.0
6,5000.0,5000.0


In [25]:
rare3_num = rare3.apply(pd.to_numeric, errors='raise')
rare3_num.head()

,ceiling_set_1
2,30000.0
3,30000.0
4,30000.0
5,30000.0
6,30000.0


For categorical feature 'Weather Condition', binarize with **dummy/indicator** variables.

In [26]:
wx_cond_set1 = set(common2.unique())

In [27]:
wx_cond_set2 = set(rare4.iloc[:,0].unique())

In [28]:
wx_cond_set3 = set(rare4.iloc[:,1].unique())

In [29]:
wx_bin = pd.get_dummies(common2)
wx_bin.head()

,Blowing Dust,Blowing Snow,Clear,Fog,Frz Rain,Hail,Haze,Heavy Rain,Heavy Rain/Thunderstorm,Heavy Snow,Ice Fog,Ice pellets,Light Frz Rain,Light Ice Pellets,Light Rain,Light Rain/Thunderstorm,Light Snow,Light Snow Grains,Light Snow Pellets,Mostly Clear,Mostly Cloudy,Overcast,Partly Cloudy,Rain,Snow,Snow Pellets,Squalls,Thunder,Thunderstorm,Unknown Precip
2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [30]:
wx_bin2 = pd.get_dummies(rare4)
wx_bin2.head()

,weather_condition_set_2d_Blowing Dust,weather_condition_set_2d_Blowing Snow,weather_condition_set_2d_Fog,weather_condition_set_2d_Haze,weather_condition_set_2d_Heavy Rain,weather_condition_set_2d_Ice Fog,weather_condition_set_2d_Light Ice Pellets,weather_condition_set_2d_Light Rain,weather_condition_set_2d_Light Snow,weather_condition_set_2d_Light Snow Pellets,weather_condition_set_2d_Light Snow Shower,weather_condition_set_2d_Rain,weather_condition_set_2d_Snow,weather_condition_set_2d_Snow Pellets,weather_condition_set_2d_Squalls,weather_condition_set_3d_Fog,weather_condition_set_3d_Hail,weather_condition_set_3d_Rain
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [31]:
wx_bin['Light Snow Shower'] = wx_bin2['weather_condition_set_2d_Light Snow Shower']

In [32]:
for wx in wx_cond_set2:
    name = str(wx)
    if(name == 'nan'):
        continue
    wx_bin[name] = wx_bin[name] | wx_bin2['weather_condition_set_2d_'+name]

for wx in wx_cond_set3:
    name = str(wx)
    if(name == 'nan'):
        continue
    wx_bin[name] = wx_bin[name] | wx_bin2['weather_condition_set_3d_'+name]

In [33]:
wx_bin.head()

,Blowing Dust,Blowing Snow,Clear,Fog,Frz Rain,Hail,Haze,Heavy Rain,Heavy Rain/Thunderstorm,Heavy Snow,Ice Fog,Ice pellets,Light Frz Rain,Light Ice Pellets,Light Rain,Light Rain/Thunderstorm,Light Snow,Light Snow Grains,Light Snow Pellets,Mostly Clear,Mostly Cloudy,Overcast,Partly Cloudy,Rain,Snow,Snow Pellets,Squalls,Thunder,Thunderstorm,Unknown Precip,Light Snow Shower
2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Concatenate Together 

In [34]:
wx = pd.concat([perfect_dt, common1_num, rare1_num, rare2_num, rare3_num, wx_bin], axis=1)

In [35]:
wx.head()

,Date_Time,air_temp_set_1,dew_point_temperature_set_1d,wind_speed_set_1,wind_direction_set_1,cloud_layer_1_code_set_1,visibility_set_1,wind_gust_set_1,snow_depth_set_1,cloud_layer_2_code_set_1,cloud_layer_3_code_set_1,ceiling_set_1,Blowing Dust,Blowing Snow,Clear,Fog,Frz Rain,Hail,Haze,Heavy Rain,Heavy Rain/Thunderstorm,Heavy Snow,Ice Fog,Ice pellets,Light Frz Rain,Light Ice Pellets,Light Rain,Light Rain/Thunderstorm,Light Snow,Light Snow Grains,Light Snow Pellets,Mostly Clear,Mostly Cloudy,Overcast,Partly Cloudy,Rain,Snow,Snow Pellets,Squalls,Thunder,Thunderstorm,Unknown Precip,Light Snow Shower
2,2016-01-01 00:05:00,12.02,5.98,7.0,360.0,1.0,10.0,0.0,0.0,5000.0,5000.0,30000.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2016-01-01 00:10:00,12.02,5.98,7.0,360.0,1.0,10.0,0.0,0.0,5000.0,5000.0,30000.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2016-01-01 00:15:00,12.02,4.88,7.0,360.0,1.0,10.0,0.0,0.0,5000.0,5000.0,30000.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,2016-01-01 00:20:00,10.94,4.90,7.0,350.0,1.0,10.0,0.0,0.0,5000.0,5000.0,30000.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,2016-01-01 00:25:00,10.94,4.90,7.0,350.0,1.0,10.0,0.0,0.0,5000.0,5000.0,30000.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Flight Data

### Read Data from CSV

In [36]:
to2016 = pd.read_csv('data/toslc.csv', index_col=0, low_memory=False)
to2017 = pd.read_csv('data/toslc2017.csv', index_col=0, low_memory=False)

In [37]:
to_raw = pd.concat([to2016, to2017])
to_raw.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,UniqueCarrier,AirlineID,Carrier,TailNum,FlightNum,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,DestState,DestStateFips,DestStateName,DestWac,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,Cancelled,CancellationCode,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,FirstDepTime,TotalAddGTime,LongestAddGTime,DivAirportLandings,DivReachedDest,DivActualElapsedTime,DivArrDelay,DivDistance,Div1Airport,Div1AirportID,Div1AirportSeqID,Div1WheelsOn,Div1TotalGTime,Div1LongestGTime,Div1WheelsOff,Div1TailNum,Div2Airport,Div2AirportID,Div2AirportSeqID,Div2WheelsOn,Div2TotalGTime,Div2LongestGTime,Div2WheelsOff,Div2TailNum,Div3Airport,Div3AirportID,Div3AirportSeqID,Div3WheelsOn,Div3TotalGTime,Div3LongestGTime,Div3WheelsOff,Div3TailNum,Div4Airport,Div4AirportID,Div4AirportSeqID,Div4WheelsOn,Div4TotalGTime,Div4LongestGTime,Div4WheelsOff,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Unnamed: 109
0,2016,1,1,1,5,2016-01-01,AA,19805,AA,N3JAAA,79,11298,1129804,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,14869,1486903,34614,SLC,"Salt Lake City, UT",UT,49,Utah,87,2230,2316.0,46.0,46.0,1.0,3.0,2200-2259,11.0,2327.0,44.0,5.0,20,49.0,29.0,29.0,1.0,1.0,0001-0559,0.0,NaN,0.0,170.0,153.0,137.0,1.0,989.0,4,0.0,0.0,0.0,0.0,29.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016,1,1,2,6,2016-01-02,AA,19805,AA,N3ALAA,79,11298,1129804,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,14869,1486903,34614,SLC,"Salt Lake City, UT",UT,49,Utah,87,2230,2231.0,1.0,1.0,0.0,0.0,2200-2259,18.0,2249.0,10.0,47.0,20,57.0,37.0,37.0,1.0,2.0,0001-0559,0.0,NaN,0.0,170.0,206.0,141.0,1.0,989.0,4,1.0,0.0,36.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016,1,1,3,7,2016-01-03,AA,19805,AA,N3EPAA,79,11298,1129804,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,14869,1486903,34614,SLC,"Salt Lake City, UT",UT,49,Utah,87,2230,2335.0,65.0,65.0,1.0,4.0,2200-2259,11.0,2346.0,58.0,12.0,20,110.0,50.0,50.0,1.0,3.0,0001-0559,0.0,NaN,0.0,170.0,155.0,132.0,1.0,989.0,4,0.0,0.0,0.0,0.0,50.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016,1,1,4,1,2016-01-04,AA,19805,AA,N3GLAA,79,11298,1129804,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,14869,1486903,34614,SLC,"Salt Lake City, UT",UT,49,Utah,87,2230,2227.0,-3.0,0.0,0.0,-1.0,2200-2259,14.0,2241.0,2349.0,6.0,20,2355.0,-25.0,0.0,0.0,-2.0,0001-0559,0.0,NaN,0.0,170.0,148.0,128.0,1.0,989.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016,1,1,5,2,2016-01-05,AA,19805,AA,N3KSAA,79,11298,1129804,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,14869,1486903,34614,SLC,"Salt Lake City, UT",UT,49,Utah,87,2230,2225.0,-5.0,0.0,0.0,-1.0,2200-2259,12.0,2237.0,2358.0,3.0,18,1.0,-17.0,0.0,0.0,-2.0,0001-0559,0.0,NaN,0.0,168.0,156.0,141.0,1.0,989.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

### Select Relevant Features

In [38]:
to_select = to_raw[['Month','DayofMonth','DayOfWeek','AirlineID','TailNum','FlightNum',
                    'OriginAirportID','CRSDepTime','FlightDate','CRSArrTime','Distance',
                    'ArrDel15']]
to_select.head()

,Month,DayofMonth,DayOfWeek,AirlineID,TailNum,FlightNum,OriginAirportID,CRSDepTime,FlightDate,CRSArrTime,Distance,ArrDel15
0,1,1,5,19805,N3JAAA,79,11298,2230,2016-01-01,20,989.0,1.0
1,1,2,6,19805,N3ALAA,79,11298,2230,2016-01-02,20,989.0,1.0
2,1,3,7,19805,N3EPAA,79,11298,2230,2016-01-03,20,989.0,1.0
3,1,4,1,19805,N3GLAA,79,11298,2230,2016-01-04,20,989.0,0.0
4,1,5,2,19805,N3KSAA,79,11298,2230,2016-01-05,18,989.0,0.0


### Drop Missing Values

In [39]:
to = to_select.dropna(axis=0, how='any')

In [40]:
to.is_copy = False #disable false positive warnings

### Feature Encoding 

Change circraft tail number to numberic.

In [41]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(to.loc[:,'TailNum'])

LabelEncoder()

In [42]:
to.loc[:,'TailNum'] = le.transform(to.loc[:,'TailNum'])

Convert departure time and arriving time to 4-digit string.

In [43]:
to.loc[:,'CRSDepTime'] = to.loc[:,'CRSDepTime'].apply(lambda x:str(x).zfill(4))
to.loc[:,'CRSArrTime'] = to.loc[:,'CRSArrTime'].apply(lambda x:str(x).zfill(4))

Create another feature "DateTime" by concatenating flight date and arring time for joining with weather data.

In [44]:
to['DateTime'] = to.loc[:,'FlightDate'].astype(str)+'T'+to.loc[:,'CRSArrTime'].astype(str)
to.drop('FlightDate', axis=1, inplace=True)

In [45]:
to.loc[:,'DateTime'] = to.loc[:,'DateTime'].apply(lambda x:
                                                  datetime.strptime(x,'%Y-%m-%dT%H%M'))

Convert departure time and arriving time to numeric.

In [46]:
to.loc[:,'CRSDepTime'] = to.loc[:,'CRSDepTime'].apply(pd.to_numeric, errors='raise')
to.loc[:,'CRSArrTime'] = to.loc[:,'CRSArrTime'].apply(pd.to_numeric, errors='raise')

In [47]:
to.head()

,Month,DayofMonth,DayOfWeek,AirlineID,TailNum,FlightNum,OriginAirportID,CRSDepTime,CRSArrTime,Distance,ArrDel15,DateTime
0,1,1,5,19805,1095,79,11298,2230,20,989.0,1.0,2016-01-01 00:20:00
1,1,2,6,19805,930,79,11298,2230,20,989.0,1.0,2016-01-02 00:20:00
2,1,3,7,19805,1020,79,11298,2230,20,989.0,1.0,2016-01-03 00:20:00
3,1,4,1,19805,1061,79,11298,2230,20,989.0,0.0,2016-01-04 00:20:00
4,1,5,2,19805,1132,79,11298,2230,18,989.0,0.0,2016-01-05 00:18:00


**[Important]**

For flights with an arriving time between 12:00am and 05:00am, add one day from the original datetime to make sure we assign the corresponding weather info to them. It is because the original date is the departure date which is the previous day of the arriving date.

In [48]:
to.loc[to.loc[:,'CRSArrTime']<500,'DateTime'] = \
to.loc[to.loc[:,'CRSArrTime']<500,'DateTime'] + timedelta(days=1)

In [49]:
to.head()

,Month,DayofMonth,DayOfWeek,AirlineID,TailNum,FlightNum,OriginAirportID,CRSDepTime,CRSArrTime,Distance,ArrDel15,DateTime
0,1,1,5,19805,1095,79,11298,2230,20,989.0,1.0,2016-01-02 00:20:00
1,1,2,6,19805,930,79,11298,2230,20,989.0,1.0,2016-01-03 00:20:00
2,1,3,7,19805,1020,79,11298,2230,20,989.0,1.0,2016-01-04 00:20:00
3,1,4,1,19805,1061,79,11298,2230,20,989.0,0.0,2016-01-05 00:20:00
4,1,5,2,19805,1132,79,11298,2230,18,989.0,0.0,2016-01-06 00:18:00


# Join 

Round-up the arriving time to every 5 minutes in order to join with weather data.

In [50]:
to.loc[:,'DateTime'] = to.loc[:,'DateTime'] \
.apply(lambda x:x - timedelta(minutes=x.minute % 5))

Join the flight operation data and weather data.

In [51]:
join = pd.merge(to, wx, left_on='DateTime', right_on='Date_Time', how='inner')
join = join.rename(columns={'ArrDel15':'temp'})
join['ArrDel15'] = join['temp']
join = join.drop(['Date_Time','temp'], axis=1)

In [52]:
data2016 = join[join['DateTime'].apply(lambda x:(x.year==2016 or 
                                                (x.year==2017 and x.month==1 and 
                                                 x.day==1 and x.hour<5)))]
data2017 = join[join['DateTime'].apply(lambda x:x.year==2017)]

In [53]:
data2016 = data2016.drop('DateTime', axis=1)
data2017 = data2017.drop('DateTime', axis=1)

# Preparation for Prediction

In [54]:
total2016 = data2016.as_matrix()
total2017 = data2017.as_matrix()

In [55]:
Xtrain = total2016[:,:-1]
trainy = total2016[:,-1]

In [56]:
Xtest = total2017[:,:-1]
testy = total2017[:,-1]

### Feature Selection

Although we preselected some features that might affect flight operation by the experience, there are also some tools like **f_classif** to help us select features furthermore to filter out bad features. Here we keep only 33 out of 51 features with $p-value\le0.1$.

In [57]:
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import SelectKBest

In [58]:
fval, pval = f_classif(Xtrain,trainy)

/Applications/anaconda3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [21 25 30 38] are constant.
  UserWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [59]:
m = (pval<0.1).sum()
m

/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in less
  """Entry point for launching an IPython kernel.


33

In [60]:
kb = SelectKBest(k=m)
kb.fit(Xtrain,trainy)
Xtrainprime = kb.transform(Xtrain)
Xtestprime = kb.transform(Xtest)
selected = kb.get_support()

/Applications/anaconda3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [21 25 30 38] are constant.
  UserWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [61]:
pd.DataFrame([fval,pval,selected], columns=data2016.columns.delete(-1), 
             index=['f-value','p-value','selected'])

,Month,DayofMonth,DayOfWeek,AirlineID,TailNum,FlightNum,OriginAirportID,CRSDepTime,CRSArrTime,Distance,air_temp_set_1,dew_point_temperature_set_1d,wind_speed_set_1,wind_direction_set_1,cloud_layer_1_code_set_1,visibility_set_1,wind_gust_set_1,snow_depth_set_1,cloud_layer_2_code_set_1,cloud_layer_3_code_set_1,ceiling_set_1,Blowing Dust,Blowing Snow,Clear,Fog,Frz Rain,Hail,Haze,Heavy Rain,Heavy Rain/Thunderstorm,Heavy Snow,Ice Fog,Ice pellets,Light Frz Rain,Light Ice Pellets,Light Rain,Light Rain/Thunderstorm,Light Snow,Light Snow Grains,Light Snow Pellets,Mostly Clear,Mostly Cloudy,Overcast,Partly Cloudy,Rain,Snow,Snow Pellets,Squalls,Thunder,Thunderstorm,Unknown Precip,Light Snow Shower
f-value,82.1896,83.3583,0.231124,196.27,13.4008,167.173,5.72725,1600.36,1254.4,215.896,5.00112,234.727,100.531,218.756,0.344841,471.942,93.5872,11.5764,108.039,45.8381,415.209,NaN,0.448108,258.51,321.972,NaN,0.289311,0.242619,0.512335,0.817829,NaN,64.4866,7.93298,3.22006,0.115976,54.1057,2.68423,286.573,NaN,0.0172267,0.0801404,0.00849359,40.8104,2.26156,5.69138,160.502,42.6078,3.39101,8.54363,2.17017,0.034456,31.743
p-value,1.25933e-19,6.97573e-20,0.630693,1.50932e-44,0.000251659,3.30576e-38,0.0167056,0,5.85724e-273,8.05019e-49,0.0253332,6.43686e-53,1.19772e-23,1.92084e-49,0.55705,2.19158e-104,3.98058e-22,0.000668229,2.71935e-25,1.29193e-11,4.26545e-92,NaN,0.503236,4.33599e-58,7.1069e-72,NaN,0.590664,0.622322,0.474132,0.365818,NaN,9.83042e-16,0.0048552,0.0727442,0.733441,1.91527e-13,0.101349,3.45742e-64,NaN,0.895577,0.777108,0.926571,1.68519e-10,0.132624,0.0170504,9.4185e-37,6.72305e-11,0.0655559,0.00346816,0.140714,0.852741,1.76485e-08
selected,True,True,False,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True,True,True,True,False,False,True,True,False,False,False,False,False,False,True,True,True,False,True,False,True,False,False,False,False,True,False,True,True,True,True,True,False,False,True


### Feature Scaling

In [62]:
from sklearn.preprocessing import QuantileTransformer
qtu = QuantileTransformer(output_distribution='uniform')

In [63]:
from sklearn.preprocessing import QuantileTransformer
qtn = QuantileTransformer(output_distribution='normal')

In [64]:
from sklearn.preprocessing import RobustScaler
rs = RobustScaler()

In [65]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

In [66]:
from sklearn.preprocessing import Normalizer
norm = Normalizer()

In [67]:
norm.fit(Xtrainprime)
trainX = norm.transform(Xtrainprime)
testX = norm.transform(Xtestprime)

# Prediction

### Predict with Decision Tree

In [68]:
from sklearn.tree import DecisionTreeClassifier

In [69]:
dtc = DecisionTreeClassifier(max_depth=4)
predy = dtc.fit(trainX, trainy).predict(testX)

In [70]:
print('Accuracy: {0:.5f}'.format((predy==testy).sum() / testy.size))

Accuracy: 0.86415


### Predict with Random Forest

In [71]:
from sklearn.ensemble import RandomForestClassifier

In [72]:
rfc = RandomForestClassifier(n_estimators=30, max_depth=20)
predy = rfc.fit(trainX, trainy).predict(testX)

In [73]:
print('Accuracy: {0:.5f}'.format((predy==testy).sum() / testy.size))

Accuracy: 0.86256


### Predict with Linear Classifier

In [74]:
from sklearn.linear_model import Perceptron

In [75]:
perc = Perceptron(max_iter=5, tol=None)
predy = perc.fit(trainX, trainy).predict(testX)

In [76]:
print('Accuracy: {0:.5f}'.format((predy==testy).sum() / testy.size))

Accuracy: 0.77731


### Predict with Naive Bayes

In [77]:
from sklearn.naive_bayes import GaussianNB

In [78]:
gnb = GaussianNB()
predy = gnb.fit(trainX, trainy).predict(testX)

In [79]:
print('Accuracy: {0:.5f}'.format((predy==testy).sum() / testy.size))

Accuracy: 0.75045


### Predict with Neural Network

In [80]:
from sklearn.neural_network import MLPClassifier

In [81]:
mlpc = MLPClassifier()
predy = mlpc.fit(trainX, trainy).predict(testX)

In [82]:
print('Accuracy: {0:.5f}'.format((predy==testy).sum() / testy.size))

Accuracy: 0.86424
